In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'
%autosave 20

Autosaving every 20 seconds


In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
import re
import time
from geopy.geocoders import Nominatim
import requests
import pandas as pd

In [7]:
meta_df = pd.DataFrame()

for page in range(0,720,30):
    place = []
    lat = []
    long = []
    
    driver = webdriver.Chrome()
    print(f"page number{page}")
    
    url = f"https://www.tripadvisor.in/Attractions-g297628-Activities-oa{page}-Bengaluru_Bangalore_District_Karnataka.html"
    
    driver.get(url)
    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()
    
    place_ratings = soup.find_all('div', class_='jVDab o W f u w JqMhy')
    place_names = soup.find_all('div', class_='XfVdV o AIbhI')
    rating_count = soup.find_all('span', class_='biGQs _P pZUbB osNWb')
    
    
    def get_location(location):
        geolocator = Nominatim(user_agent=f"my_app_{page}")
        # Use geocode to get the coordinates of the location
        location_info = geolocator.geocode(location)
        # Retrieve latitude and longitude
        latitude = location_info.latitude
        longitude = location_info.longitude

        return latitude,longitude
    
    for div,svg,r_count in zip(place_names,place_ratings,rating_count):
        location = f"{div.text.split('.')[1]}"
        print(f"Place name {div.text.split('.')[1]} and rating {svg.find('svg')['aria-label']} Rating Count : {r_count.text.strip()}")
        
        place.append(div.text.split('.')[1])
        rating = [svg.find('svg')['aria-label'] for svg in place_ratings]
        rating_counts = [cc.text.strip() for cc in rating_count ]
        
        try:
            latitude,longitude = get_location(location)
            print(f"latitude :{latitude},longitude : {longitude} ")
            lat.append(latitude)
            long.append(longitude)
        except:
            lat.append(0)
            long.append(0)
            # latitude,longitude = get_loc2 (location)
            print("location not found")
            # print(f"latitude :{latitude},longitude : {longitude} ")
    level2 = pd.DataFrame({"place":place,'Rating': rating,'Popularity':rating_counts,'latitude':lat,'longitude':long})
    meta_df = pd.concat([meta_df,level2])

page number0
Place name  ISKCON Temple Bangalore and rating 4.5 of 5 bubbles Rating Count : 4,837
latitude :15.40417305,longitude : 75.06941224495768 
Place name  Lalbagh Botanical Garden and rating 4.0 of 5 bubbles Rating Count : 6,190
latitude :12.9489367,longitude : 77.58698687897964 
Place name  Wonderla Amusement Park and rating 4.5 of 5 bubbles Rating Count : 4,250
latitude :17.216339599999998,longitude : 78.52694354065062 
Place name  UB City and rating 4.5 of 5 bubbles Rating Count : 2,802
latitude :12.9723336,longitude : 77.59611365584038 
Place name  Bannerghatta National Park and rating 4.0 of 5 bubbles Rating Count : 2,947
latitude :12.53290595,longitude : 77.54619068920488 
Place name  Cubbon Park and rating 4.0 of 5 bubbles Rating Count : 2,497
latitude :12.97424405,longitude : 77.59219502919655 
Place name  Visvesvaraya Industrial and Technological Museum and rating 4.5 of 5 bubbles Rating Count : 1,766
latitude :12.97528365,longitude : 77.59628434047809 
Place name  Com

In [9]:
meta_df

,place,Rating,Popularity,latitude,longitude
0,ISKCON Temple Bangalore,4.5 of 5 bubbles,"4,837",15.404173,75.069412
1,Lalbagh Botanical Garden,4.0 of 5 bubbles,"6,190",12.948937,77.586987
2,Wonderla Amusement Park,4.5 of 5 bubbles,"4,250",17.216340,78.526944
3,UB City,4.5 of 5 bubbles,"2,802",12.972334,77.596114
4,Bannerghatta National Park,4.0 of 5 bubbles,"2,947",12.532906,77.546191
...,...,...,...,...,...
25,O2 Spa,3.5 of 5 bubbles,3,12.970624,77.599082
26,O2 Spa,4.0 of 5 bubbles,2,12.970624,77.599082
27,Footworks,4.0 of 5 bubbles,2,52.528187,-1.840972
28,Vriddhi,4.0 of 5 bubbles,2,12.929056,77.591476


In [11]:
meta_df.shape
meta_df.columns

(718, 5)

Index(['place', 'Rating', 'Popularity', 'latitude', 'longitude'], dtype='object')

In [13]:
meta_df.sample(3)

,place,Rating,Popularity,latitude,longitude
6,GoodShotz Photography,4.0 of 5 bubbles,3,0.000000,0.000000
2,Sanctum,4.0 of 5 bubbles,21,-27.387760,152.968692
19,St,4.5 of 5 bubbles,174,50.060158,13.830748


In [19]:
meta_df1 = meta_df[meta_df.latitude!=0].reset_index(drop = True)

In [22]:
meta_df1.shape

(288, 5)

In [24]:
meta_df1.dtypes

place          object
Rating         object
Popularity     object
latitude      float64
longitude     float64
dtype: object

In [26]:
meta_df1.Popularity = meta_df1.Popularity.apply(lambda x : x.replace(',',''))

In [27]:
meta_df1.Rating = meta_df1.Rating.apply(lambda x : x.split('of')[0])

In [28]:
meta_df1.dtypes

place          object
Rating         object
Popularity     object
latitude      float64
longitude     float64
dtype: object

In [29]:
meta_df1.sample(2)

,place,Rating,Popularity,latitude,longitude
200,Bengaluru Lights,5.0,5,12.965254,77.595615
67,Ramakrishna Math,4.5,25,22.575690,88.396121


In [30]:
meta_df1.Rating = meta_df1.Rating.astype('float')
meta_df1.Popularity = meta_df1.Popularity.astype('float')

In [31]:
meta_df1.sample(2)

,place,Rating,Popularity,latitude,longitude
36,Kaikondrahalli Lake,4.5,138.0,12.913187,77.672202
246,Body Raaga,4.5,4.0,12.970168,77.638227


In [33]:


def get_state_name(latitude, longitude):
    geolocator = Nominatim(user_agent="my_app")  # Specify a user agent for the geocoding service
    location = geolocator.reverse((latitude, longitude), exactly_one=True)
    address = location.raw['address']
    state = address.get('state', '')
    return state

In [35]:
states = []
for lat,log in zip(meta_df1.latitude,meta_df1.longitude):
    s = get_state_name(lat,log)
    states.append(s)
meta_df1['State'] = states

In [36]:
meta_df1.to_csv('Final_location_data.csv',index = False)